#### Import libraries

In [2]:
import pandas as pd
import chart_studio as cs
cs.tools.set_credentials_file(username='macv97', api_key='rW2ViBF5BhcWxasj9Ijb')
import plotly.graph_objects as go
import pandas as pd

from get_database_data import get_pessoa
from get_database_data import get_produto
from get_database_data import get_fabricante
from get_database_data import get_saida
from get_database_data import get_saida_produto

from title_treatment import *
from datetime import datetime

from descriptive_analysis import *

#### Import vocabulary

In [3]:
from vocabulary import cores
from vocabulary import marcas
from vocabulary import preposicoes
from vocabulary import abb
from vocabulary import models
from vocabulary import categories

# Feature Engineering

## Filtering the product titles

In [4]:
df = get_produto().reset_index(drop=True).drop(['PRECO_VENDA', 'CUSTO_TOTAL', 'GANHO_LIQUIDO'], axis=1)
#df = add_ith_name([1, 2, 3], df, 'NOME')
df.rename(columns={'ID': 'ID_PRODUTO'}, inplace=True)
fab = get_fabricante()
df = filter_name(df, fab, cores, abb, marcas, models, preposicoes)

dict_of_categories = categories
df.rename(columns={'NOME_FILTRADO': 'GRUPO_PRODUTO'}, inplace=True)

df = detect_category(df, 'GRUPO_PRODUTO', dict_of_categories)
#df = cluster_items(df, 'NOME_FILTRADO', list(range(1, 300, 50)), 0)
#df = cluster_items(df, 'NOME_FILTRADO', list(range(1, 300, 50)), 80)

df.head(10)


,ID_PRODUTO,NOME,ID_FABRICANTE,COR,GRUPO_PRODUTO,MODELO,CATEGORIA
0,1,ESPIRAL 07MM PRETO COM 100 UN - 25 FOLHAS,3,PRETO,ESPIRAL,07MM 100 UN - 25 FOLHAS,SUPRIMENTO
1,2,ESPIRAL 07MM CRISTAL COM 100 UN - 25 FOLHAS,3,CRISTAL,ESPIRAL,07MM 100 UN - 25 FOLHAS,SUPRIMENTO
2,3,ESPIRAL 07MM AZUL ROYAL COM 100 UN - 25 FOLHAS,3,AZUL ROYAL,ESPIRAL,07MM 100 UN - 25 FOLHAS,SUPRIMENTO
3,4,ESPIRAL 09MM AZUL ROYAL COM 100 UN - 50 FOLHAS,3,AZUL ROYAL,ESPIRAL,09MM 100 UN - 50 FOLHAS,SUPRIMENTO
4,5,ESPIRAL 09MM CRISTAL COM 100 UN - 50 FOLHAS,3,CRISTAL,ESPIRAL,09MM 100 UN - 50 FOLHAS,SUPRIMENTO
5,6,ESPIRAL 09MM PRETO COM 100 UN - 50 FOLHAS,3,PRETO,ESPIRAL,09MM 100 UN - 50 FOLHAS,SUPRIMENTO
6,7,ESPIRAL 12MM CRISTAL COM 100 UN - 70 FOLHAS,3,CRISTAL,ESPIRAL,12MM 100 UN - 70 FOLHAS,SUPRIMENTO
7,87,CAPA PP LINE CRISTAL A4 COM 100 UNID,10,CRISTAL,CAPA PP,LINE A4 100 UNID,SUPRIMENTO
8,88,CAPA PP LINE CRISTAL OFICIO COM 100 UNID,10,CRISTAL,CAPA PP,LINE OFICIO 100 UNID,SUPRIMENTO
9,89,CAPA PP LINE CRISTAL A3 COM 100 UNID,5,CRISTAL,CAPA PP,LINE A3 100 UNID,SUPRIMENTO


### Distribution of quantity of existing products per category

In [5]:
count = df['CATEGORIA'].value_counts().to_list()

bar = go.Figure(go.Bar(
    x=['SUPRIMENTOS', 'MATERIAL DE ESCRITORIO', 'EQUIPAMENTO'],
    y=count,
    marker=dict(color= 'rgba(100, 159, 237, 1)')))

bar.layout.update(title=dict(text="QUANTIDADE DE PRODUTOS POR CATEGORIA"), title_font=dict(size=18),
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  xaxis=dict(title="CATEGORIA", title_standoff=10),
                  yaxis=dict(title="# DE PRODUTOS"),
                  width = 600, height=500)
bar.show()

## Clustering products

### Clustering products by their selling rank

RETRIEVE DATA

In [6]:
# RETRIEVE TABLE SAIDA
df_saida = get_saida().reset_index(drop=True)

# RENAME COLUMN ID TO ID_SAIDA FOR BETTER DISCERNMENT 
df_saida.rename(columns={'ID': 'ID_SAIDA'}, inplace=True)

# SORT VALUES IN DESCENDING ORDER TO GET THE LAST DATE OF SELLING
df_saida = df_saida.sort_values(by='DATA', ascending=False)
last_sell = df_saida.sort_values(by='DATA', ascending=False).iloc[0]['DATA']

# RETRIEVE TABLE SAIDA_PRODUTO
df_saida_produto = get_saida_produto().reset_index(drop=True)

# RENAME COLUMN ID TO ID_SAIDAPRODUTO FOR BETTER DISCERNMENT
df_saida_produto.rename(columns={'ID': 'ID_SAIDAPRODUTO'}, inplace=True)

IMPORT NECESSARY LIBRARIES AND SHAPE THE DATAFRAME

In [7]:
# IMPORT NECESSARY LIBRARIES TO DEAL WITH DATETIME DATA
from datetime import date

#data_i = last_sell - relativedelta(months=months)
data_i = date(2017, 1, 1)
data_f = last_sell

# MERGE TABLES SAIDA AND SAIDAPRODUTO
df_vendas = pd.merge(df_saida, df_saida_produto, on='ID_SAIDA', how='outer')

# DROP CLIENTE BALCAO SHOPPING'S ROW
df_vendas.drop(df_vendas.index[df_vendas['ID_CLIENTE'] == 0], inplace=True)

# DROP CANCELED SHOPPINGS
df_vendas.drop(df_vendas.index[df_vendas['CANCELADO'] == 'S'], inplace=True)

# DROP UNNECESSARY COLUMNS
df_vendas = df_vendas.drop(['HORA', 'CANCELADO'], axis=1)

# DROP SHOPPINGS FROM BEFORE 2 YEARS AGO
df_vendas.drop(df_vendas.index[df_vendas['DATA'] < data_i], inplace=True)

# TRANSFORM VALUES IN ID_PRODUTO AND ID_SAIDAPRODUTO TO INTEGERS
df_vendas['ID_PRODUTO'] = df_vendas['ID_PRODUTO'].astype(int)
df_vendas['ID_SAIDAPRODUTO'] = df_vendas['ID_SAIDAPRODUTO'].astype(int)

# CREATE DATAFRAME FOR TIMEFRAME SELLING ANALYSIS
df_vendas = pd.merge(df_vendas, df, on='ID_PRODUTO', how='outer')
df_vendas = df_vendas.dropna().reset_index(drop=True)
#df_vendas = df_vendas[['NOME', 'GRUPO_PRODUTO', 'QUANT', 'ID_PRODUTO', 'CATEGORIA', 'PRECO', 'TOTAL']].reset_index(drop=True)
df_vendas = df_vendas.drop(['ID_SAIDA', 'ID_CLIENTE', 'TOTALPRODUTOS', 'ID_SAIDAPRODUTO','PRECO', 'ID_FABRICANTE', 'MODELO'], axis=1)

In [8]:
df_vendas

,DATA,ID_PRODUTO,QUANT,TOTAL,NOME,COR,GRUPO_PRODUTO,CATEGORIA
0,2023-05-17,5,1,25.85,ESPIRAL 09MM CRISTAL COM 100 UN - 50 FOLHAS,CRISTAL,ESPIRAL,SUPRIMENTO
1,2023-05-15,5,1,25.45,ESPIRAL 09MM CRISTAL COM 100 UN - 50 FOLHAS,CRISTAL,ESPIRAL,SUPRIMENTO
2,2023-05-10,5,1,26.5,ESPIRAL 09MM CRISTAL COM 100 UN - 50 FOLHAS,CRISTAL,ESPIRAL,SUPRIMENTO
3,2023-04-28,5,1,25.04,ESPIRAL 09MM CRISTAL COM 100 UN - 50 FOLHAS,CRISTAL,ESPIRAL,SUPRIMENTO
4,2023-04-27,5,1,26.5,ESPIRAL 09MM CRISTAL COM 100 UN - 50 FOLHAS,CRISTAL,ESPIRAL,SUPRIMENTO
...,...,...,...,...,...,...,...,...
131752,2017-04-04,1143,1,15,TAXA DE ENTREGA,,TAXA ENTREGA,None
131753,2017-04-03,1143,1,15,TAXA DE ENTREGA,,TAXA ENTREGA,None
131754,2017-03-31,455,2,124,TONER BROTHER CART. TN 450/DCP7066 LASER TONER,,TONER,SUPRIMENTO
131755,2017-03-30,1012,1,190,ROLO DE PRESSAO RICOH AF1035/1045 PAULIMAC,,ROLO PRESSAO,SUPRIMENTO


### Produtos mais vendidos

In [9]:
dict_weekdays = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}

In [10]:
# DEFINE TIME PERIOD TO BE CONSIDERED IN OUR ANALYSIS
from dateutil.relativedelta import relativedelta

initial_date = date(2017, 1, 1)
final_date = date(2023, 5, 17)

time_step = final_date - initial_date
time_difference = relativedelta(final_date, initial_date)

diff_months = time_difference.years * 12 + time_difference.months
diff_days = time_step.days
diff_weeks = diff_days//7

step = 'week'
years = 2
month_intervals = list(range(diff_months, -1, -1))
week_intervals = list(range(years*72, -1, -1))
day_intervals = list(range(diff_days, -1, -1))
last_sell = df_vendas.sort_values(by='DATA', ascending=False).iloc[0]['DATA']
df_vendas = define_time_periods(df_vendas, last_sell, week_intervals, step)

### In the last two years

In [11]:
produtos_mais_vendidos, df_timeframe = top_n_sales(df, df_vendas, 24,  last_sell)
show_top_sales(produtos_mais_vendidos, 'TWO YEARS', 10)

## Frequency of sales per product 

### Polaseal
To analyze the frequency of sales per product I will start by analysing the sales of the polaseal, the top source of revenue in the last year

In [12]:
df_polaseal = product_over_type(df_vendas, 'POLASEAL')
plot_product_over_type_dist(df_vendas, 'POLASEAL')

In [14]:
product = 'ESPIRAL'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step, 'PRETO')
plot_product_over_year(df_product, product, step)

c:\Users\macvi\Documents\AI\Data Science\juliana\descriptive_analysis.py:247: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



KeyError: 'year'

In [13]:
product = 'MATERIAL_DE_ESCRITORIO'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
df_revenue = revenue_over_year(df_vendas, product, step)
list_of_dfs = [df_product, df_revenue]
plot_over_year(list_of_dfs, product, step)

In [11]:
product = 'ALL'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
plot_product_over_year(df_product, product, step)

In [12]:
product = 'POLASEAL'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
plot_product_over_year(df_product, product, step)

In [31]:
product = 'GARRA'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
plot_product_over_year(df_product, product, step)

In [32]:
product = 'ESPIRAL'
color = 'PRETO'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
plot_product_over_year(df_product, product, step)

In [33]:
product = 'TONER'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
plot_product_over_year(df_product, product, step)

In [34]:
product = 'CAPA PP'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
plot_product_over_year(df_product, product, step)

In [35]:
product = 'PLASTIFICADORA'
step = 'week'

df_product, df_dates = product_over_year(df_vendas, product, step)
plot_product_over_year(df_product, product, step)

## Time Series Decomposition

## Client-specific buying behavior

Let us now run through a couple of examples of client buying histories. We are interested in visually investigating whether we can spot patterns of purchase from specific buyers.  